# Markov generator workbook

In [33]:
from __future__ import annotations
import generator
from IPython.display import *
from IPython.lib.pretty import pretty
import pandas

class ObservationNode:
    def __init__(self, pattern) -> None:
        self.pattern = pattern
        self.count = 0
        self.children = {}

    def _repr_html_(self) -> str:
        out = "<b>{} ({})</b><ul>".format(self.pattern, self.count)
        children = sorted(self.children.keys(), key=lambda x: -self.children[x].count) 
        for key in children :
            out += "<li>" + self.children[key]._repr_html_() + "</li>"
        out += "</ul>"
        return out 

    def findOrCreate(self, pattern) -> ObservationNode:
        next = self
        pat = ""
        for ch in pattern:
            pat = pat + ch
            if pat not in next.children :
                next.children[pat] = ObservationNode(pat)
            next = next.children[pat]
        return next
    
    def addNextObservation(self, pattern) -> ObservationNode:
        if (pattern not in self.children):
            self.children[pattern] = ObservationNode(pattern)
        next = self.children[pattern]
        next.count = next.count + 1
        return next

class MarkovModel:
    def __init__(self, depth = 3) -> None:
        self.tree = ObservationNode("")
        self.data = "Undefined"
        self.depth = depth

    def resetObservationState(self) :
        self.pattern = ""
        self.currentObservation = self.tree

    def trainFromFile(self, fileName) -> None:
        trainingFile = open(fileName)
        for line in trainingFile:
            self.resetObservationState()
            for ch in line:
                self.observe(ch)

    def observe(self, ch) :
        if ch >= 'A' and ch <='Z':
            ch = ch.lower()
        if ch >= 'a' and ch <='z':
            if (len(self.pattern) == self.depth) :
                self.pattern = self.pattern[1:]
                self.currentObservation = self.tree.findOrCreate(self.pattern)
            self.pattern = self.pattern + ch
            self.currentObservation = self.currentObservation.addNextObservation(self.pattern)
        else:
            self.resetObservationState()


In [34]:
model = MarkovModel()
model.trainFromFile("../Training/training.txt")

display(model.tree)